# Amazon Fine Food Reviews Analysis


Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews <br>

EDA: https://nycdatascience.com/blog/student-works/amazon-fine-foods-visualization/


The Amazon Fine Food Reviews dataset consists of reviews of fine foods from Amazon.<br>

Number of reviews: 568,454<br>
Number of users: 256,059<br>
Number of products: 74,258<br>
Timespan: Oct 1999 - Oct 2012<br>
Number of Attributes/Columns in data: 10 

Attribute Information:

1. Id
2. ProductId - unique identifier for the product
3. UserId - unqiue identifier for the user
4. ProfileName
5. HelpfulnessNumerator - number of users who found the review helpful
6. HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not
7. Score - rating between 1 and 5
8. Time - timestamp for the review
9. Summary - brief summary of the review
10. Text - text of the review


#### Objective:
Given a review, determine whether the review is positive (Rating of 4 or 5) or negative (rating of 1 or 2).

<br>
[Q] How to determine if a review is positive or negative?<br>
<br> 
[Ans] We could use the Score/Rating. A rating of 4 or 5 could be cosnidered a positive review. A review of 1 or 2 could be considered negative. A review of 3 is nuetral and ignored. This is an approximate and proxy way of determining the polarity (positivity/negativity) of a review.




# Downloading the data using the Kaggle API

In [ ]:
! ls

sample_data


In [ ]:
! pip install -q kaggle

In [ ]:
! pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 3.6MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.10-cp36-none-any.whl size=73269 sha256=d6d37064c18b0f8d9d34cfa80177322689cf37158e4f551403cd5e240f6a9fe4
  Stored in directory: /root/.cache/pip/wheels/3a/d1/7e/6ce09b72b770149802c653a02783821629146983ee5a360f10
Successfully built kaggle
  Found existing installation: kaggle 1.5.10
    Uninstalling kaggle-1.5.10:
      Successfully uninstalled kaggle-1.5.10


In [ ]:
from google.colab import files

In [ ]:
files.upload()

NameError: name 'files' is not defined

In [ ]:
! mkdir ~/.kaggle

In [ ]:
cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list 

ref                                                            title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
utkarshxy/who-worldhealth-statistics-2020-complete             World Health 2020 🌏 | For Geospatial Analysis         1MB  2021-01-06 16:22:50           1021        140  1.0              
gpreda/pfizer-vaccine-tweets                                   Pfizer Vaccine Tweets                               416KB  2021-01-07 12:08:56            757         77  1.0              
google/android-smartphones-high-accuracy-datasets              Android smartphones high accuracy GNSS datasets       1GB  2020-12-23 01:51:11            143         31  0.875            
ashkhagan/women-representation-in-city-property-sanfrancisco   Wo

In [ ]:
! pip install --upgrade --force-reinstall --no-deps kaggle

Processing /root/.cache/pip/wheels/3a/d1/7e/6ce09b72b770149802c653a02783821629146983ee5a360f10/kaggle-1.5.10-cp36-none-any.whl
  Found existing installation: kaggle 1.5.10
    Uninstalling kaggle-1.5.10:
      Successfully uninstalled kaggle-1.5.10


In [ ]:
! kaggle datasets download -d snap/amazon-fine-food-reviews

 96% 233M/242M [00:02<00:00, 126MB/s]
100% 242M/242M [00:02<00:00, 122MB/s]


In [ ]:
! ls

amazon-fine-food-reviews.zip  kaggle.json  sample_data


In [ ]:
! unzip amazon-fine-food-reviews.zip 

Archive:  amazon-fine-food-reviews.zip
  inflating: Reviews.csv             
  inflating: database.sqlite         
  inflating: hashes.txt              


In [ ]:
! ls

amazon-fine-food-reviews.zip  gdrive	  kaggle.json  sample_data
database.sqlite		      hashes.txt  Reviews.csv


## Loading the data

The dataset is available in two forms
1. .csv file
2. SQLite Database

In order to load the data, We have used the SQLITE dataset as it easier to query the data and visualise the data efficiently.
<br> 

Here as we only want to get the global sentiment of the recommendations (positive or negative), we will purposefully ignore all Scores equal to 3. If the score id above 3, then the recommendation wil be set to "positive". Otherwise, it will be set to "negative".

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")



import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

import re
from bs4 import BeautifulSoup

# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

# Modeling packages
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# mod_file = 'lr_bow.model'
# pickle.dump(mod_file, open('/content/gdrive/My Drive/my_model.pkl', 'wb'))


In [ ]:
df = pd.read_csv('Reviews.csv')
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
df['sentiment'] = np.where(df['Score']>3, 1, 0)
df = df[df['Score'] != 3]


In [ ]:
df['sentiment'].value_counts()

1    443777
0     82037
Name: sentiment, dtype: int64

In [ ]:
# df['Text'].iloc[0]

In [ ]:
eng_stop_words = stopwords.words('english')
# eng_stop_words

In [ ]:
noise_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",
               "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his',
               'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself',
               'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this',
               'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been',
               'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an',
               'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by',
               'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before',
               'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off',
               'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where',
               'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some',
               'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's',
               't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll',
               'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn',
               "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't",
               'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't",
               'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won',
               "won't", 'wouldn', "wouldn't"]

In [ ]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can't", "can not", phrase)

    # in the above line both will work
    # phrase = re.sub(r"won't", "will not", phrase) and phrase = re.sub(r"won\'t", "will not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:

def clean_text(text):
  # global stop_words
  text = text.lower() 
  text = re.sub(r"http\S+", "", text)
  text = BeautifulSoup(text, 'lxml').get_text()
  text = decontracted(text)
  text = re.sub('\S*\d\S*', ' ', text).strip()
  text = re.sub('[^A-Za-z]+', ' ', text) 
  # text = ' '.join(e.lower() for e in text.split() if e not in noise_words)  
  # text = ' '.join(word for word in word_tokenize(text) if word not in stop_words)
  return text.strip()
  

In [ ]:
df['Text'] = df['Text'].apply(lambda x: clean_text(x))

In [ ]:
# The following code creates a word-document matrix.
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
X = vec.fit_transform(df['Text'])


In [ ]:
### Creating a python object of the class CountVectorizer

bow_counts = CountVectorizer(tokenizer= word_tokenize, # type of tokenization
                             stop_words=noise_words, # List of stopwords
                             ngram_range=(1,2)) # number of n-grams

bow_data = bow_counts.fit_transform(df['Text'])

# Save the vectorizer
# vec_file = 'bow_counts.pickle'


In [ ]:
pickle.dump(bow_counts, open('/content/gdrive/My Drive/bow_counts.pickle', 'wb'))

In [ ]:
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(bow_data, df['sentiment'], test_size = 0.3, random_state = 42)

In [ ]:
lr_bow = LogisticRegression()
lr_bow.fit(X_train_bow, y_train_bow)

test_pred_lr_bow = lr_bow.predict(X_test_bow)

# Save the model
# mod_file = 'lr_bow.model'


## Calculate key performance metrics
print("F1 score: ", f1_score(y_test_bow, test_pred_lr_bow))

F1 score:  0.9786680229775012


In [ ]:
pickle.dump(lr_bow, open('/content/gdrive/My Drive/lr_bow.model', 'wb'))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_counts = TfidfVectorizer(tokenizer= word_tokenize,
                               stop_words = noise_words, 
                               ngram_range=(1,2))

tfidf_data = tfidf_counts.fit_transform(df['Text'])

# Save the vectorizer
# vec_file = 'tfidf_counts.pickle'


In [ ]:
pickle.dump(tfidf_counts, open('/content/gdrive/My Drive/tfidf_counts.pickle', 'wb'))

In [ ]:
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(tfidf_data, df['sentiment'], test_size = 0.3,random_state = 42)


In [ ]:
lr_tf_idf = LogisticRegression()

lr_tf_idf.fit(X_train_tfidf,y_train_tfidf)

test_pred_lr_tf_idf = lr_tf_idf.predict(X_test_tfidf)

# Save the model
# mod_file = 'lr_tf_idf.model'


## Evaluating the model
print("F1 score: ",f1_score(y_test_bow, test_pred_lr_tf_idf))

F1 score:  0.9739700000739279


In [ ]:
pickle.dump(lr_tf_idf, open('/content/gdrive/My Drive/lr_tf_idf.model', 'wb'))